### 주식 정보 가져오기
- url = https://finance.naver.com/item/sise_day.nhn?code=003490&page=1
- url 로부터 html 가져오기 requests
- 가져온 html로부터 분석하기 BeautifulSoup
- 해당객체의 정보 가져오기

In [18]:
import requests
from bs4 import BeautifulSoup
import cx_Oracle

In [19]:
# 변수지정
uid = 'PROJECT1'
upw = 'Test1234'
url = 'localhost:1521/orcl'
conn = cx_Oracle.connect(uid,upw,url)
cursor=conn.cursor()
#print(conn)
url = 'https://finance.naver.com/item/sise_day.nhn?code=003490&page=1'

In [20]:
#data = [20200812, 19250, 20100, 19050, 19550, 8846552]
def insertStock(data=[]):
    sql = "insert into STOCKNAV (SIDX,SDATE,OPENP,HIGHP,LOWP,CLOSEP,VOL,SSTAT) values(STOCK_SEQ.NEXTVAL,{0},{1},{2},{3},{4},{5},0)"
    sql1= sql.format(*data)
    
    try:
        cursor.execute(sql1)
        return(1)
    except cx_Oracle.DatabaseError as exc:
        error, = exc.args
        print("Oracle-Error-Code:", error.code)
        print("Oracle-Error-Message:", error.message)
        return(0)
    
#insertStock(data)

In [31]:
#data=[20200811, 19250, 20100, 19050, 19550, 8846552]
def addData(data):
    sql="select * from STOCKNAV where SDATE="+str(data[0])
    cursor.execute(sql)
    res=cursor.fetchall()
    if(len(res)>0):
        print("데이터 중복")
    else:
        #print("입력가능 ")
        insertStock(data)
        conn.commit()
        print(data[0], " 입력 완료")
        
#addData(data)

In [32]:
# ohlc(open-high-low-close)
def setData(src):
    for d in src:
        tgt = d.select('td')
        n=len(d.select('td.num'))
        if(n>0):
            sdate = tgt[0].get_text().replace(".","")
            endp = tgt[1].get_text().replace(",","")
            startp = tgt[3].get_text().replace(",","")
            maxp = tgt[4].get_text().replace(",","")
            minp = tgt[5].get_text().replace(",","")
            vol = tgt[6].get_text().replace(",","")
            t = [int(sdate), int(startp), int(maxp), int(minp), int(endp), int(vol)]
            addData(t)
            print(t)
            


In [33]:
def myNaverStock(url):
    req=requests.get(url)
    res=req.text[:100]
    if(len(res)>50): # 많이 긁으면 서버가 경고 메세지 날릴수도 있다. 경고 메세지(대략 50자) 이상 들어와야지 시작
        html=req.text
        soup=BeautifulSoup(html,'html.parser')
        src=soup.select('table.type2 > tr')
        setData(src)

In [38]:
baseUrl = 'https://finance.naver.com/item/sise_day.nhn?code=003490&page='
for i in range(500,600):
    url=baseUrl+str(i)
    myNaverStock(url)

20000530  입력 완료
[20000530, 6810, 7350, 6810, 7050, 1407460]
20000529  입력 완료
[20000529, 6110, 6800, 6030, 6690, 798300]
20000526  입력 완료
[20000526, 6890, 7000, 6400, 6400, 1161350]
20000525  입력 완료
[20000525, 6400, 7060, 6390, 7060, 2292900]
20000524  입력 완료
[20000524, 5550, 6400, 5530, 6140, 1630170]
20000523  입력 완료
[20000523, 5300, 5850, 5300, 5600, 1048670]
20000522  입력 완료
[20000522, 5850, 5950, 5520, 5520, 736280]
20000519  입력 완료
[20000519, 5950, 6080, 5680, 6020, 1110560]
20000518  입력 완료
[20000518, 6000, 6100, 5810, 6000, 1252740]
20000515  입력 완료
[20000515, 7800, 7980, 7050, 7050, 1326660]
20000512  입력 완료
[20000512, 8700, 8750, 8050, 8120, 1674880]
20000510  입력 완료
[20000510, 8900, 9050, 8620, 8680, 2130260]
20000509  입력 완료
[20000509, 9400, 9450, 8960, 9100, 1180350]
20000508  입력 완료
[20000508, 9500, 9700, 9310, 9400, 655230]
20000504  입력 완료
[20000504, 9610, 9720, 9500, 9570, 460560]
20000503  입력 완료
[20000503, 9850, 9850, 9600, 9800, 321850]
20000502  입력 완료
[20000502, 10000, 10200, 9950